In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import toolz as fp

In [16]:
## Constants
abreviacao = ['RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS',
              'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP']
descricao = ['Robadas de Bola', 'Faltas Cometidas', 'Gol Contra', 'Cartao Amarelo',
             'Cartao Vermelho', 'Jogo sem sofrer gol', 'Defesa Dificil', 'Defesa de Penalti',
             'Gol Sofrido', 'Falta Sofrida', 'Passe Errado', 'Assistencia',
             'Finalizacao na Trave', 'Finalizacao defendida', 'Finalizao pra Fora', 'Gol',
             'Impedimento', 'Penalti Perdido']
pontuacao = [1.7, -0.5, -6.0, -2.0, -5.0, 5.0, 3.00, 7.00, -2.0,
             0.5, -0.3, 5.0, 3.5, 1.0, 0.7, 8.0, -0.5, -3.5]

points_dict = {
    "RB": 1.7,
    "FC": -0.5,
    "GC": -6.0,
    "CA": -2.0,
    "CV": -5.0,
    "SG": 5.0,
    "DD": 3.0,
    "DP": 7.0,
    "GS": -2.0,
    "FS": 0.5,
    "PE": -0.3,
    "A": 5.0,
    "FT": 3.5,
    "FD": 1.0,
    "FF": 0.7,
    "G": 8.0,
    "I": -0.5,
    "PP": -3.5,
}

team_dict = {
'265':'BAH',
'288':'CRI',
'267':'VAS',
'290':'GOI',
'314':'AVA',
'317':'JEC',
'262':'FLA',
'263':'BOT',
'264':'COR',
'266':'FLU',
'275':'PAL',
'276':'SAO',
'277':'SAN',
'282':'CAM',
'283':'CRU',
'284':'GRE',
'285':'INT',
'287':'VIT',
'292':'SPT',
'293':'CAP',
'294':'CFC',
'303':'PON',
'315':'CHA',
'316':'FIG',
'327':'AME',
'344':'SCZ',
'373':'ATL-GO'
}

posicoes = ["gol", "lat", "zag",  "mei", "ata", "tec"]

status = {
		"2": "Dúvida",
		"3": "Suspenso",
		"5": "Contundido",
		"6": "Nulo",
		"7": "Provável"
	}


In [92]:
df = pd.read_csv('../db/cartola_aggregated_with_results.csv').rename(columns={'Unnamed: 0': 'ScoutID'})

In [93]:
df.head()

,ScoutID,AtletaID,Rodada,ClubeID,Participou,Posicao,Jogos,Pontos,PontosMedia,Preco,...,away_score,pred.home.score,pred.away.score,home.attack,home.defend,variable,ClubeCasa,ClubeVisitante,Home,result
0,0,36443,1,285.0,True,gol,1.0,5.0,5.0,10.60,...,0.0,0.986759,1.03473,0.076288,-0.438249,home.team,285,287,True,1
1,1,36540,1,285.0,True,zag,1.0,2.0,2.0,12.88,...,0.0,0.986759,1.03473,0.076288,-0.438249,home.team,285,287,True,1
2,2,37655,1,285.0,True,ata,1.0,0.5,0.5,6.44,...,0.0,0.986759,1.03473,0.076288,-0.438249,home.team,285,287,True,1
3,3,38074,1,285.0,True,mei,1.0,1.7,1.7,3.56,...,0.0,0.986759,1.03473,0.076288,-0.438249,home.team,285,287,True,1
4,4,38108,1,285.0,True,ata,1.0,0.2,0.2,10.00,...,0.0,0.986759,1.03473,0.076288,-0.438249,home.team,285,287,True,1


In [8]:
print(list(df.columns))

['Unnamed: 0', 'AtletaID', 'Rodada', 'ClubeID', 'Participou', 'Posicao', 'Jogos', 'Pontos', 'PontosMedia', 'Preco', 'PrecoVariacao', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS', 'ano', 'Apelido', 'Status', 'avg.Points', 'avg.last05', 'avg.FS', 'avg.FS.l05', 'avg.PE', 'avg.PE.l05', 'avg.A', 'avg.A.l05', 'avg.FT', 'avg.FT.l05', 'avg.FD', 'avg.FD.l05', 'avg.FF', 'avg.FF.l05', 'avg.G', 'avg.G.l05', 'avg.I', 'avg.I.l05', 'avg.PP', 'avg.PP.l05', 'avg.RB', 'avg.RB.l05', 'avg.FC', 'avg.FC.l05', 'avg.GC', 'avg.GC.l05', 'avg.CA', 'avg.CV.l05', 'avg.SG', 'avg.SG.l05', 'avg.DD', 'avg.DD.l05', 'avg.DP', 'avg.DP.l05', 'avg.GS', 'avg.GS.l05', 'risk_points', 'mes', 'dia', 'home_score', 'away_score', 'pred.home.score', 'pred.away.score', 'home.attack', 'home.defend', 'variable', 'ClubeCasa', 'ClubeVisitante', 'Home', 'result']


In [10]:
df.shape

(39404, 82)

In [11]:
df.drop(['pred.home.score', 'pred.away.score', 'home.attack', 'home.defend', 'variable'],
        axis=1, inplace=True)

In [13]:
auxiliary = ['AtletaID', 'Rodada', 'ClubeID', 'Participou', 'Posicao', 'ano', 'mes', 'dia', 'Apelido', 'Status']

In [83]:
plays = ['FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 
         'CV', 'SG', 'DD', 'DP', 'GS']

In [67]:
df['game_points'] = df.apply(lambda x: 3 if ((x['result'] == 1 and x['Home']) or (x['result'] == -1 and not x['Home']))
                        else 1 if x['result'] == 0 else 0, axis=1)

In [60]:
df.loc[100].ClubeVisitante

277

## Get Info ScoutID

In [63]:
def getAtletaID(df, scout_id):
    return df.loc[scout_id].AtletaID

def getClubeID(df, scout_id):
    return df.loc[scout_id].ClubeID

def getClubeVisitante(df, scout_id):
    return df.loc[scout_id].ClubeVisitante

def getClubeCasa(df, scout_id):
    return df.loc[scout_id].ClubeCasa

def getRodada(df, scout_id):
    return df.loc[scout_id].Rodada

def getPosicao(df, scout_id):
    return df.loc[scout_id].Posicao

def getAno(df, scout_id):
    return df.loc[scout_id].ano

def getHome(df, scout_id):
    return df.loc[scout_id].Home

def getResult(df, scout_id):
    return df.loc[scout_id].result



## Filters

In [80]:
def filterDfLastRounds(df, rodada, ano, num_rounds):
    return df.loc[(((df.ano == ano) & (df.Rodada < rodada) & (df.Rodada >= rodada - num_rounds)) 
                                                | ((df.ano == ano - 1) & (df.Rodada >= 38 + rodada - num_rounds)))]

def filterDfClube(df, clubeID):
    return df.loc[df.ClubeID == clubeID]

def filterHome(df, home):
    return df.loc[df.Home == home]

def filterDfPosicao(df, pos):
    return df.loc[df.Posicao == pos]

def filterDfResult(df, res):
    return df.loc[df.result == res]

## Custom Aggregations

## Aggregate

In [84]:

def getColumnAverageLastRounds(df, scout_id, column_name, num_rounds):
    atleta_id = getAtletaID(df, scout_id)
    rodada = getRodada(df, scout_id)
    ano = getAno(df, scout_id)
    return filterDfLastRounds(df, rodada, ano, num_rounds)[column_name].mean()

def getColumnStdDevLastRounds(df, scout_id, column_name, num_rounds):
    atleta_id = getAtletaID(df, scout_id)
    rodada = getRodada(df, scout_id)
    ano = getAno(df, scout_id)
    return filterDfLastRounds(df, rodada, ano, num_rounds)[column_name].std()

def getColumnSumLastRounds(df, scout_id, column_name, num_rounds):
    atleta_id = getAtletaID(df, scout_id)
    rodada = getRodada(df, scout_id)
    ano = getAno(df, scout_id)
    return filterDfLastRounds(df, rodada, ano, num_rounds)[column_name].sum()

def getColumnCountLastRounds(df, scout_id, column_name, num_rounds):
    atleta_id = getAtletaID(df, scout_id)
    rodada = getRodada(df, scout_id)
    ano = getAno(df, scout_id)
    return filterDfLastRounds(df, rodada, ano, num_rounds)[column_name].count()


In [42]:
getColumnStdDevLastRounds(df, 100, 'FF', 10)

0.7867957924694431

In [51]:
filterDfLastRounds(df, 20, 2016, 10).Rodada.value_counts()

17    297
11    297
16    297
14    297
12    297
13    296
15    294
10    293
19    269
18    268
Name: Rodada, dtype: int64

## Team Goals

In [58]:
def getTeamGolsScored(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_home = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == True)]
    df_away = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == False)]
    df_team = df.loc[(df_filtered.ClubeID == clube)]
    sum_goals_home = getColumnSumLastRounds(df_home, scout_id, "home_score", num_rounds)
    sum_goals_away = getColumnSumLastRounds(df_away, scout_id, "away_score", num_rounds)
    count_games = getColumnCountLastRounds(df_team, scout_id, "home_score", num_rounds)
    return (sum_goals_home + sum_goals_away) / count_games

def getTeamGolsTaken(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_home = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == True)]
    df_away = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == False)]
    df_team = df.loc[(df_filtered.ClubeID == clube)]
    sum_goals_home = getColumnSumLastRounds(df_home, scout_id, "away_score", num_rounds)
    sum_goals_away = getColumnSumLastRounds(df_away, scout_id, "home_score", num_rounds)
    count_games = getColumnCountLastRounds(df_team, scout_id, "home_score", num_rounds)
    return (sum_goals_home + sum_goals_away) / count_games

def getTeamGolsScoredHome(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == True)]
    return getColumnAverageLastRounds(df_filtered, scout_id, "home_score", num_rounds)

def getTeamGolsScoredAway(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == False)]
    return getColumnAverageLastRounds(df_filtered, scout_id, "away_score", num_rounds)

def getTeamGolsTakenHome(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == True)]
    return getColumnAverageLastRounds(df_filtered, scout_id, "away_score", num_rounds)

def getTeamGolsTakenAway(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == False)]
    return getColumnAverageLastRounds(df_filtered, scout_id, "home_score", num_rounds)

In [89]:
df.apply(lambda x: x["Home"], axis=1)

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
         ...  
39374    False
39375    False
39376    False
39377    False
39378    False
39379    False
39380    False
39381    False
39382    False
39383    False
39384    False
39385    False
39386    False
39387    False
39388    False
39389    False
39390    False
39391    False
39392    False
39393    False
39394    False
39395    False
39396    False
39397    False
39398    False
39399    False
39400    False
39401    False
39402    False
39403    False
Length: 39404, dtype: bool

## Enemy Goals

In [64]:
def getEnemyGolsScored(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_home = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == True)]
    df_away = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == False)]
    df_team = df.loc[(df_filtered.ClubeID == clube)]
    sum_goals_home = getColumnSumLastRounds(df_home, scout_id, "home_score", num_rounds)
    sum_goals_away = getColumnSumLastRounds(df_away, scout_id, "away_score", num_rounds)
    count_games = getColumnCountLastRounds(df_team, scout_id, "home_score", num_rounds)
    return (sum_goals_home + sum_goals_away) / count_games

def getEnemyGolsTaken(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_home = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == True)]
    df_away = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == False)]
    df_team = df.loc[(df_filtered.ClubeID == clube)]
    sum_goals_home = getColumnSumLastRounds(df_home, scout_id, "away_score", num_rounds)
    sum_goals_away = getColumnSumLastRounds(df_away, scout_id, "home_score", num_rounds)
    count_games = getColumnCountLastRounds(df_team, scout_id, "home_score", num_rounds)
    return (sum_goals_home + sum_goals_away) / count_games

def getEnemyGolsScoredHome(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == True)]
    return getColumnAverageLastRounds(df_filtered, scout_id, "home_score", num_rounds)

def getEnemyGolsScoredAway(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == False)]
    return getColumnAverageLastRounds(df_filtered, scout_id, "away_score", num_rounds)

def getEnemyGolsTakenHome(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == True)]
    return getColumnAverageLastRounds(df_filtered, scout_id, "away_score", num_rounds)

def getEnemyGolsTakenAway(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = df.loc[(df_filtered.ClubeID == clube) & (df_filtered.Home == False)]
    return getColumnAverageLastRounds(df_filtered, scout_id, "home_score", num_rounds)

In [ ]:
for num_rounds in [1, 5, 10, 15]:
    for play in plays + ['Pontos'] + ['Preco']:
        df['avg_' + play + 'last_' + str(num_rounds)] = df.apply(
            lambda x: getColumnAverageLastRounds(df, x.ScoutID, play, num_rounds))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getTeamGolsScored(df, x.ScoutID, play))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getTeamGolsTaken(df, x.ScoutID, play))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getTeamGolsScoredHome(df, x.ScoutID, play))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getTeamGolsScoredAway(df, x.ScoutID, play))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getTeamGolsTakenHome(df, x.ScoutID, play))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getTeamGolsTakenAway(df, x.ScoutID, play))
        
        
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getEnemyGolsScored(df, x.ScoutID, play))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getEnemyGolsTaken(df, x.ScoutID, play))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getEnemyGolsScoredHome(df, x.ScoutID, play))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getEnemyGolsScoredAway(df, x.ScoutID, play))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getEnemyGolsTakenHome(df, x.ScoutID, play))
        
    df['team_goals_home_last_' + str(num_rounds)] = df.apply(
        lambda x: getEnemyGolsTakenAway(df, x.ScoutID, play))

## Team/Enemy Goals Ratio

In [65]:
def getTeamScoredEnemyTakenRatio(df, scout_id, num_rounds):
    return getTeamGolsScored(df, scout_id, num_rounds) / getEnemyGolsTaken(df, scout_id, num_rounds)

def getTeamScoredEnemyTakenRatioHome(df, scout_id, num_rounds):
    return getTeamGolsScoredHome(df, scout_id, num_rounds) / getEnemyGolsTakenAway(df, scout_id, num_rounds)

def getTeamScoredEnemyTakenRatioAway(df, scout_id, num_rounds):
    return getTeamGolsScoredAway(df, scout_id, num_rounds) / getEnemyGolsTakenHome(df, scout_id, num_rounds)


def getTeamTakenEnemyScoredRatio(df, scout_id, num_rounds):
    return getTeamGolsTaken(df, scout_id, num_rounds) / getEnemyGolsScored(df, scout_id, num_rounds)

def getTeamTakenEnemyScoredRatioHome(df, scout_id, num_rounds):
    return getTeamGolsTakenHome(df, scout_id, num_rounds) / getEnemyGolsScoredAway(df, scout_id, num_rounds)

def getTeamTakenEnemyScoredRatioAway(df, scout_id, num_rounds):
    return getTeamGolsTakenAway(df, scout_id, num_rounds) / getEnemyGolsScoredHome(df, scout_id, num_rounds)

## Team Players Points

In [72]:
def getAverageTeamPoints(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getAverageTeamPointsHome(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), True)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getAverageTeamPointsAway(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), False)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getAverageEnemyPoints(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getAverageEnemyPointsHome(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), True)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getAverageEnemyPointsAway(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), False)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getAverageTeamPointsEnemyRatio(df, scout_id, num_rounds):
    return getAverageTeamPoints(df, scout_id, num_rounds) / getAverageEnemyPoints(df, scout_id, num_rounds)

def getAverageTeamPointsEnemyRatioHome(df, scout_id, num_rounds):
    return getAverageTeamPointsHome(df, scout_id, num_rounds) / getAverageEnemyPointsAway(df, scout_id, num_rounds)

def getAverageTeamPointsEnemyRatio(df, scout_id, num_rounds):
    return getAverageTeamPointsAway(df, scout_id, num_rounds) / getAverageEnemyPointsHome(df, scout_id, num_rounds)



In [70]:
def getStdTeamPoints(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getStdTeamPointsHome(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), True)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getStdTeamPointsAway(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), False)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getStdEnemyPoints(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getStdEnemyPointsHome(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), True)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getStdEnemyPointsAway(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), False)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getStdTeamPointsEnemyRatio(df, scout_id, num_rounds):
    return (getAverageTeamPoints(df, scout_id, num_rounds) - 
            getAverageEnemyPoints(df, scout_id, num_rounds)) / getStdTeamPoints(df, scout_id, num_rounds)

def getStdTeamPointsEnemyRatioHome(df, scout_id, num_rounds):
    return (getAverageTeamPointsHome(df, scout_id, num_rounds) - 
            getAverageEnemyPointsAway(df, scout_id, num_rounds)) / getStdTeamPointsHome(df, scout_id, num_rounds)

def getStdTeamPointsEnemyRatioAway(df, scout_id, num_rounds):
    return (getAverageTeamPointsAway(df, scout_id, num_rounds) - 
            getAverageEnemyPointsHome(df, scout_id, num_rounds)) / getStdTeamPointsAway(df, scout_id, num_rounds)



## Team Game Points

In [ ]:
def getAverageTeamGamePoints(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnAverageLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getAverageTeamGamePointsHome(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), True)
    return getColumnAverageLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getAverageTeamGamePointsAway(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), False)
    return getColumnAverageLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getAverageEnemyGamePoints(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnAverageLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getAverageEnemyGamePointsHome(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), True)
    return getColumnAverageLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getAverageEnemyGamePointsAway(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), False)
    return getColumnAverageLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getAverageTeamGamePointsEnemyRatio(df, scout_id, num_rounds):
    return getAverageTeamPoints(df, scout_id, num_rounds) / getAverageEnemyPoints(df, scout_id, num_rounds)

def getAverageTeamGamePointsEnemyRatioHome(df, scout_id, num_rounds):
    return getAverageTeamGamePointsHome(df, scout_id, num_rounds) / getAverageEnemyGamePointsAway(df, scout_id, num_rounds)

def getAverageTeamGamePointsEnemyRatio(df, scout_id, num_rounds):
    return getAverageTeamGamePointsAway(df, scout_id, num_rounds) / getAverageEnemyGamePointsHome(df, scout_id, num_rounds)



In [76]:
def getStdTeamGamePoints(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getStdTeamGamePointsHome(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), True)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getStdTeamGamePointsAway(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), False)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getStdEnemyGamePoints(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getStdEnemyGamePointsHome(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), True)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getStdEnemyGamePointsAway(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterHome(filterDfClube(df, clube), False)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "game_points", num_rounds)

def getStdTeamGamePointsEnemyRatio(df, scout_id, num_rounds):
    return (getAverageTeamGamePoints(df, scout_id, num_rounds) - 
            getAverageEnemyGamePoints(df, scout_id, num_rounds)) / getStdTeamGamePoints(df, scout_id, num_rounds)

def getStdTeamGamePointsEnemyRatioHome(df, scout_id, num_rounds):
    return (getAverageTeamGamePointsHome(df, scout_id, num_rounds) - 
            getAverageEnemyGamePointsAway(df, scout_id, num_rounds)) / getStdTeamGamePointsHome(df, scout_id, num_rounds)

def getStdTeamGamePointsEnemyRatioAway(df, scout_id, num_rounds):
    return (getAverageTeamGamePointsAway(df, scout_id, num_rounds) - 
            getAverageEnemyGamePointsHome(df, scout_id, num_rounds)) / getStdTeamGamePointsAway(df, scout_id, num_rounds)



## Team Price

In [74]:
def getAverageTeamPrice(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Preco", num_rounds)

def getAverageEnemyPrice(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Preco", num_rounds)

def getAverageTeamEnemyPriceRatio(df, scout_id, num_rounds):
    return getAverageTeamPrice(df, scout_id, num_rounds) / getAverageEnemyPrice(df, scout_id, num_rounds)

In [73]:
def getStdTeamPrice(df, scout_id, num_rounds):
    clube = getClubeID(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Preco", num_rounds)

def getStdEnemyPrice(df, scout_id, num_rounds):
    clube = getClubeVisitante(df, scout_id) if getHome(df, scout_id) else getClubeCasa(df, scout_id)
    df_filtered = filterDfClube(df, clube)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Preco", num_rounds)

def getStdTeamEnemyPriceRatio(df, scout_id, num_rounds):
    return (getAverageTeamPrice(df, scout_id, num_rounds) - 
            getAverageEnemyPrice(df, scout_id, num_rounds)) / getStdTeamPrice(df, scout_id, num_rounds)

# Position Features

## Average Points Position

In [78]:
def getAveragePositionPoints(df, scout_id, num_rounds):
    pos = getPosicao(df, scout_id)
    df_filtered = filterDfPosicao(df, pos)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getAveragePositionPointsHome(df, scout_id, num_rounds):
    pos = getPosicao(df, scout_id)
    df_filtered = filterHome(filterDfPosicao(df, pos), True)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getAveragePositionPointsAway(df, scout_id, num_rounds):
    pos = getPosicao(df, scout_id)
    df_filtered = filterHome(filterDfPosicao(df, pos), False)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)
    

## StdDev Points Position

In [79]:
def getStdPositionPoints(df, scout_id, num_rounds):
    pos = getPosicao(df, scout_id)
    df_filtered = filterDfPosicao(df, pos)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getStdPositionPointsHome(df, scout_id, num_rounds):
    pos = getPosicao(df, scout_id)
    df_filtered = filterHome(filterDfPosicao(df, pos), True)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getStdPositionPointsAway(df, scout_id, num_rounds):
    pos = getPosicao(df, scout_id)
    df_filtered = filterHome(filterDfPosicao(df, pos), False)
    return getColumnStdDevLastRounds(df_filtered, scout_id, "Pontos", num_rounds)
    

## Average Points Position V/E/D

In [82]:
def getAveragePositionPointsWin(df, scout_id, num_rounds):
    pos = getPosicao(df, scout_id)
    df_filtered = filterDfResult(filterDfPosicao(df, pos), 1)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getAveragePositionPointsDraw(df, scout_id, num_rounds):
    pos = getPosicao(df, scout_id)
    df_filtered = filterDfResult(filterDfPosicao(df, pos), 0)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)

def getAveragePositionPointsLoss(df, scout_id, num_rounds):
    pos = getPosicao(df, scout_id)
    df_filtered = filterDfResult(filterDfPosicao(df, pos), -1)
    return getColumnAverageLastRounds(df_filtered, scout_id, "Pontos", num_rounds)
    